*Note: before configuring your data please refer to [Map Template](../Export/MapTemplate) Notebook.*

<p style="font-weight:bold;"> <span style="font-size: 36px"> Close Import Template</span> </p>

All imports are triggered via the <code>Import</code> command. Several options can be fluently appended to customize the import action:
- <code>FromFile()</code> : accepts a string as input (eg. "../Files/TransactionalData/Openings_CH_2020_12.csv"). It corresponds the file to be imported complete with the path
- <code>WithFormat()</code> : accepts a string as input (eg. "Cashflow" or "Actual"). It is used to trigger the desired import logic for the current file. Full list of avalaible ImportFormats can be found [here](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.2/Constants/Consts#import-formats)
- <code>WithFileStorage()</code> : defined the drive from where the input files is retrieved. In this example we use the project file storage (new files have to be uploaded into the project), other options are SharePoint and OneDrive. 
- <code>WithTarget()</code> : target drive where the results are stored. In this example we use our in-memory set up


# Set up data and configuration

The data source is initialized and the whole IFRS 17 Calculation Engine code is compiled. 

Choose to run the Close Import Template notebook either on top of the set of Systemorph data in memory or on top to the data present in the Database: 
- #!eval-notebook "../Database/Configure" : connects to a physical Database
- #!eval-notebook "../Initialization/InitSystemorphToMemory" : uses the in-memory set up

We use here the in-memory set up.

In [0]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

# DataNode and Parameters

New Data Nodes and Parameters are imported here.
<br> To easily generate these files, please refer to the **Map Template** files exported [here](../Export/MapTemplate). Edit them at your wish and import them back into the solution through the following cells. 
<br> For the **ImportFormat**, the following options are expected:
- ImportFormats.DataNode : [Portfolios](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.2/DataModel/DataStructure#portfolios) and [Group of Contracts](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.2/DataModel/DataStructure#group-of-contracts)
- ImportFormats.DataNodeState : the [state of a data node](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.2/DataModel/DataStructure#data-node-state) can be either active or inactive.
- ImportFormats.DataNodeParameter : parameters are described [here](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.2/DataModel/DataStructure#data-node-parameters). For **Group of Insurance Contracts** a default [Premium Allocation factor](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.2/Import/4ImportScope-TechnicalMargin#experience-adjustment-on-premium) of 1 is applied if the parameter is not imported.


In [0]:
var pathToFile = "../Files/DataNodes/DataNodes_CH.csv";

In [0]:
var format = ImportFormats.DataNode;

In [0]:
await Import.FromFile(pathToFile)
            .WithFormat(format)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()

# Transactional Data

Transactional data are imported here. 
<br> For the **ImportFormat**, the following options are expected:
- ImportFormats.Opening : for opening values in a Fair Value Approach
- ImportFormats.Cashflow : nominal cash flows
- ImportFormats.Actual : written actuals, payable and receivables. 
- ImportFormats.SimpleValue : direct import of pre-calculated Present Value and Actual values for all reports - no pre-calculation at import is triggered, only at report level -

The results are visible in the [Report](../Report/Reports) notebook.

In [0]:
var pathToFile = "../Files/TransactionalData/Openings_CH_2020_12.csv";

In [0]:
var format = ImportFormats.Opening;

In [0]:
await Import.FromFile(pathToFile)
            .WithFormat(format)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()

# Workspace reset

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
await DataSource.Partition.SetAsync<PartitionByReportingNode>(null);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(null);